In [1]:
from transformers import AutoTokenizer, AutoConfig, GPTJForCausalLM

In [2]:
import json

with open('/home/ubuntu/server2/hansard/hansard-qa.json') as fopen:
    hansard = json.load(fopen)

In [3]:
hansard[0]

[{'original': '\n22                                                            DR.30.10.2018                                                                                                                          \n\n \ndan sebagainya. Cukai-cukai yang didapati daripada minuman keras, alkohol dan sebagainya \nmasuk kepada kerajaan, kerajaan bayar gaji kita balik, itu pun haram pada kita.  [Disampuk]  \n\nTimbalan Yang di-Pertua [Dato’ Mohd Rashid Hasnon]: Yang Berhormat Padang \nSerai, sila ajukan soalan. \n\nTuan Karupaiya Mutusami [Padang Serai]: Saya bertanya soalan pada pihak \nkementerian, bolehkah samsu dihapuskan terus, tidak mahu ada samsu dalam negara kita. \n[Dewan tepuk] Terima kasih. \n\nTuan Che Alias bin Hamid [Kemaman]: Sokong, sokong. \n\n■1150 \n\nTuan Chong Chieng Jen: Itu satu isu atau soalan yang begitu serius dan effect-\nnya yang sangat luas. Saya rasa ini perlu dibawa ke dalam Jemaah Menteri untuk \ndipertimbangkan dan diputuskan. Sekian, terima kasih. \n\n \n\n

In [4]:
json.loads(hansard[0][1])

{'soalan': [{'soalan': 'Adakah samsu akan dihapuskan terus dari negara kita?',
   'jawapan': 'Ini perlu dibawa ke dalam Jemaah Menteri untuk dipertimbangkan dan diputuskan.'},
  {'soalan': 'Adakah kerajaan bercadang untuk menyemak semula jadual perjalanan bagi kereta api komuter dan ETS dari Gemas ke Kuala Lumpur?',
   'jawapan': 'KTMB pada masa ini masih belum ada perancangan untuk menyemak semula jadual perjalanan bagi perkhidmatan komuter dan ETS daripada Gemas ke Kuala Lumpur.'},
  {'soalan': 'Berapa banyak set ETS yang dimiliki oleh KTMB untuk tujuan operasi?',
   'jawapan': 'KTMB memiliki 15 set train ETS yang aktif untuk tujuan operasi.'}]}

In [5]:
template = {
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}

In [6]:
import random
from tqdm import tqdm

In [7]:
import malaya

model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3', force_check = False)

In [8]:
_ = model.cuda()

In [9]:
model.generate(["Apakah makna nama 'Guinness'?"],
              max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
              num_return_sequences=5)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


["Apa makna nama 'Guinness'?",
 "Apakah maksud nama 'Guinness' itu?",
 "Apa maksud nama 'Guinness'?",
 "Apa makna nama 'Guinness' tu",
 "Apa makna nama 'Guinness' tu"]

In [10]:
data_parlimen = []
for d in tqdm(hansard):
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        if random.random() > 0.5:
            q = t['original']
        else:
            q = t['cleaned']
        
        
        for r in j_a['soalan']:
            randint = random.randint(0, 3)
            if random.random() > 0.5:
                generated = model.generate(
                    [r['soalan']],
                    max_length = 256, 
                    do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                    num_return_sequences=5,
                )
                soalan = random.choice(generated)
            else:
                soalan = r['soalan']
            
            if random.random() > 0.5:
                soalan = soalan.lower()
            
            if random.random() > 0.7:
                q_ = f"paragraph `{q}`, berdasarkan paragraph, jawab soalan `{soalan}`"
            else:
                q_ = f"paragraph `{q}`, {soalan}"
            
            res = template["prompt_no_input"].format(instruction=q_)
            res = f"{res}{r['jawapan']}"
            data_parlimen.append(res)
        
        k = '({"soalan": [{`soalan`, `jawapan`}]})'
        p = f"""
paragraph, `{t['cleaned']}`
hasilkan 3 soalan dan jawapan khusus berdasarkan perenggan itu, kembalikan sebagai struktur JSON {k}
        """
        res = template["prompt_no_input"].format(instruction=p)
        res = f"{res}{a}"
        data_parlimen.append(res)

    except Exception as e:
        print(e)
        pass

  5%|████▎                                                                                        | 1945/42440 [06:58<1:33:36,  7.21it/s]

Unterminated string starting at: line 12 column 17 (char 764)


  6%|█████▊                                                                                       | 2678/42440 [09:38<2:16:39,  4.85it/s]

Expecting ',' delimiter: line 13 column 109 (char 1293)


  7%|██████▌                                                                                      | 3013/42440 [10:51<2:14:21,  4.89it/s]

...

 95%|███████████████████████████████████████████████████████████████████████████████████████▊    | 40519/42440 [2:26:32<09:01,  3.55it/s]

Unterminated string starting at: line 13 column 18 (char 1779)


 98%|█████████████████████████████████████████████████████████████████████████████████████████▋  | 41389/42440 [2:29:23<02:25,  7.22it/s]

Expecting ',' delimiter: line 13 column 332 (char 1032)


100%|████████████████████████████████████████████████████████████████████████████████████████████| 42440/42440 [2:32:26<00:00,  4.64it/s]


In [11]:
len(data_parlimen)

169482

In [12]:
data_parlimen[0]

"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nparagraph `dan sebagainya. Cukai-cukai yang didapati daripada minuman keras, alkohol dan sebagainya masuk kepada kerajaan, kerajaan bayar gaji kita balik, itu pun haram pada kita. [Disampuk] Timbalan Yang di-Pertua [Dato' Mohd Rashid Hasnon]: Yang Berhormat Padang Serai, sila ajukan soalan. Tuan Karupaiya Mutusami [Padang Serai]: Saya bertanya soalan pada pihak kementerian, bolehkah samsu dihapuskan terus, tidak mahu ada samsu dalam negara kita. [Dewan tepuk] Terima kasih. Tuan Che Alias bin Hamid [Kemaman]: Sokong, sokong. #1150 Tuan Chong Chieng Jen: Itu satu isu atau soalan yang begitu serius dan effect- nya yang sangat luas. Saya rasa ini perlu dibawa ke dalam Jemaah Menteri untuk dipertimbangkan dan diputuskan. Sekian, terima kasih. 9. Dr. Hasan bin Bahrom [Tampin] minta Menteri Pengangkutan menyatakan adakah kerajaan bercadang untuk menyemak semula jadua

In [13]:
data_parlimen[6]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nparagraph `terhad, maka saya ingin memberikan jawapan bertulis tentang peruntukan mengenai seksyen 375, apakah sebenarnya requirement seksyen 375 itu. Jadi, saya rasa marital rape ini, pihak kerajaan tidak akan menjadikan satu kesalahan kerana ia amat sukar untuk dibuktikan di mahkamah. Sebagai pengamal undang- undang yang agak lama menjadi peguam, saya mengatakan bahawa agak sukar untuk membuktikan marital rape ini. Sekiranya badan mahkamah berkemungkinan untuk dilepaskan lebih mudah daripada untuk disabitkan. Oleh sebab itulah, kerajaan mengambil kira semua faktor ini. Sebagai kerajaan saya rasa belum masanya mungkin untuk menjadikan ia sebagai satu kesalahan. Akan tetapi bagaimanapun kalau ada kes-kes yang melibatkan mungkin kekerasan dalam rumah tangga, seksyen 375A akan digunakan untuk memastikan hak-hak wanita dijamin oleh pihak kerajaan. Sekian, terima 

In [14]:
with open('prepared-parliment.json', 'w') as fopen:
    json.dump(data_parlimen, fopen)